In [ ]:
# ------------------------------------------------------------------------
# a2a_agents.py
# ------------------------------------------------------------------------
import logging
import os

import abc
from collections.abc import AsyncIterable
from typing import TYPE_CHECKING, Annotated, Any, Literal

from pydantic import BaseModel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings,
    AzureChatCompletion
)
from semantic_kernel.contents import (
    FunctionCallContent,
    FunctionResultContent,
    StreamingChatMessageContent,
    StreamingTextContent,
)
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments

from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase

import asyncio
from semantic_kernel.connectors.mcp import MCPSsePlugin

if TYPE_CHECKING:
    from semantic_kernel.contents import ChatMessageContent

logger = logging.getLogger(__name__)

# region Response Format


class ResponseFormat(BaseModel):
    """A Response Format model to direct how the model should respond."""

    status: Literal['input_required', 'completed', 'error'] = 'input_required'
    message: str


# endregion

class AbstractAgent(abc.ABC):
    """
    A minimal, implementation-agnostic contract for any assistant agent using any framework.

    Concrete subclasses may wrap Semantic Kernel, LangChain, your own
    in-house stack, or even a local LLM – as long as they satisfy this API.
    """

    #: MIME types that downstream code can rely on receiving.
    SUPPORTED_CONTENT_TYPES: list[str] = ['text', 'text/plain']

    # ------------------------------------------------------------------ #
    #  Lifecycle helpers
    # ------------------------------------------------------------------ #

    async def __aenter__(self):
        # subclasses may override; by default do nothing
        return self

    async def __aexit__(self, exc_type, exc, tb):
        # subclasses may override; by default do nothing
        return False                     # propagate any exception

    @abc.abstractmethod
    async def invoke(self, user_input: str, session_id: str) -> dict[str, Any]:  # noqa: D401
        """
        Handle a *single-shot* request.

        Implementations **must** be idempotent: calling twice with the same
        `(user_input, session_id)` pair should yield the same logical answer,
        even if the underlying LLM re-generates new text.
        """

# region Semantic Kernel Agent


class SemanticKernelAgent(AbstractAgent):
    """Wraps Semantic Kernel-based agents to handle tasks."""

    # agent: ChatCompletionAgent
    # thread: ChatHistoryAgentThread = None
    # mcp_plugin: MCPSsePlugin = None
    # mcp_url: str = None


    SUPPORTED_CONTENT_TYPES = ['text', 'text/plain']

    def __init__(self, mcp_url: str, title: str,
                 oai_client: ChatCompletionClientBase):
        # just stash config – DO NOT build anything heavy here
        self._mcp_url   = mcp_url.rstrip('/')
        self._title     = title
        self._oai_client = oai_client

        # runtime attributes populated in __aenter__
        self.mcp_plugin: MCPSsePlugin | None = None
        self.agent:      ChatCompletionAgent | None = None
        self.thread:     ChatHistoryAgentThread | None = None

    # ------------------------------------------------------------------
    # async context-manager wires the plugin correctly
    async def __aenter__(self) -> "SemanticKernelAgent":
        # 1. open the SSE plugin
        self.mcp_plugin = MCPSsePlugin(
            name        = self._title,
            url         = self._mcp_url,
            description = f"{self._title} Plugin",
        )
        await self.mcp_plugin.__aenter__()            # <-- crucial

        # 2. build the SK agent (note the **singular** `plugin=`)
        self.agent = ChatCompletionAgent(
            service = self._oai_client,
            name    = f"{self._title}_agent",
            instructions = (
                f"You are a helpful assistant for {self._title} queries."
            ),
            plugins  = [self.mcp_plugin],    
            arguments = KernelArguments(
                settings = OpenAIChatPromptExecutionSettings(
                    response_format = ResponseFormat,
                )
            ),
        )
        return self

    async def __aexit__(self, exc_type, exc, tb):
        if self.thread:
            await self.thread.delete()
            self.thread = None
        if self.mcp_plugin:
            await self.mcp_plugin.__aexit__(exc_type, exc, tb)
        return False
    

    # ------------------------------------------------------------------
    async def invoke(self, user_input: str, session_id: str) -> dict[str, Any]:
        """Handle synchronous tasks (like tasks/send).

        Args:
            user_input (str): User input message.
            session_id (str): Unique identifier for the session.

        Returns:
            dict: A dictionary containing the content, task completion status, and user input requirement.
        """
        await self._ensure_thread_exists(session_id)

        # Use SK's get_response for a single shot
        response = await self.agent.get_response(
            messages=user_input,
            thread=self.thread,
        )
        return self._get_agent_response(response.content)

    async def stream(
        self,
        user_input: str,
        session_id: str,
    ) -> AsyncIterable[dict[str, Any]]:
        """For streaming tasks we yield the SK agent's invoke_stream progress.

        Args:
            user_input (str): User input message.
            session_id (str): Unique identifier for the session.

        Yields:
            dict: A dictionary containing the content, task completion status,
            and user input requirement.
        """
        await self._ensure_thread_exists(session_id)

        plugin_notice_seen = False
        plugin_event = asyncio.Event()

        text_notice_seen = False
        chunks: list[StreamingChatMessageContent] = []

        async def _handle_intermediate_message(
            message: 'ChatMessageContent',
        ) -> None:
            """Handle intermediate messages from the agent."""
            nonlocal plugin_notice_seen
            if not plugin_notice_seen:
                plugin_notice_seen = True
                plugin_event.set()
            # An example of handling intermediate messages during function calling
            for item in message.items or []:
                if isinstance(item, FunctionResultContent):
                    print(
                        f'############ Function Result:> {item.result} for function: {item.name}'
                    )
                elif isinstance(item, FunctionCallContent):
                    print(
                        f'############ Function Call:> {item.name} with arguments: {item.arguments}'
                    )
                else:
                    print(f'############ Message:> {item}')

        async for chunk in self.agent.invoke_stream(
            messages=user_input,
            thread=self.thread,
            on_intermediate_message=_handle_intermediate_message,
        ):
            if plugin_event.is_set():
                yield {
                    'is_task_complete': False,
                    'require_user_input': False,
                    'content': 'Processing function calls...',
                }
                plugin_event.clear()

            if any(isinstance(i, StreamingTextContent) for i in chunk.items):
                if not text_notice_seen:
                    yield {
                        'is_task_complete': False,
                        'require_user_input': False,
                        'content': 'Building the output...',
                    }
                    text_notice_seen = True
                chunks.append(chunk.message)

        if chunks:
            yield self._get_agent_response(sum(chunks[1:], chunks[0]))

    def _get_agent_response(
        self, message: 'ChatMessageContent'
    ) -> dict[str, Any]:
        """Extracts the structured response from the agent's message content.

        Args:
            message (ChatMessageContent): The message content from the agent.

        Returns:
            dict: A dictionary containing the content, task completion status, and user input requirement.
        """
        structured_response = ResponseFormat.model_validate_json(
            message.content
        )

        default_response = {
            'is_task_complete': False,
            'require_user_input': True,
            'content': 'We are unable to process your request at the moment. Please try again.',
        }

        if isinstance(structured_response, ResponseFormat):
            response_map = {
                'input_required': {
                    'is_task_complete': False,
                    'require_user_input': True,
                },
                'error': {
                    'is_task_complete': False,
                    'require_user_input': True,
                },
                'completed': {
                    'is_task_complete': True,
                    'require_user_input': False,
                },
            }

            response = response_map.get(structured_response.status)
            if response:
                return {**response, 'content': structured_response.message}

        return default_response

    async def _ensure_thread_exists(self, session_id: str) -> None:
        """Ensure the thread exists for the given session ID.

        Args:
            session_id (str): Unique identifier for the session.
        """
        if self.thread is None or self.thread.id != session_id:
            await self.thread.delete() if self.thread else None
            self.thread = ChatHistoryAgentThread(thread_id=session_id)


# endregion


In [ ]:
# ------------------------------------------------------------------------
# a2a_agent_exec.py
# ------------------------------------------------------------------------
import logging

from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events.event_queue import EventQueue
from a2a.types import (
    TaskArtifactUpdateEvent,
    TaskState,
    TaskStatus,
    TaskStatusUpdateEvent,
)
from a2a.utils import (
    new_agent_text_message,
    new_data_artifact,
    new_task,
    new_text_artifact,
)

# from a2a_agents import AbstractAgent
from typing_extensions import override


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class A2ALabAgentExecutor(AgentExecutor):
    """ Agent Executor """

    def __init__(self, agent: AbstractAgent):
        super().__init__()
        self.agent = agent

    # ---------------- async context-manager glue -------------
    async def __aenter__(self):
        if hasattr(self.agent, "__aenter__"):
            await self.agent.__aenter__()
        return self

    async def __aexit__(self, exc_type, exc, tb):
        if hasattr(self.agent, "__aexit__"):
            await self.agent.__aexit__(exc_type, exc, tb)
        return False
    # ---------------------------------------------------------

    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        query = context.get_user_input()
        task = context.current_task
        if not task:
            task = new_task(context.message)
            event_queue.enqueue_event(task)

        async for partial in self.agent.stream(query, task.contextId):
            require_input = partial['require_user_input']
            is_done = partial['is_task_complete']
            text_content = partial['content']

            if require_input:
                event_queue.enqueue_event(
                    TaskStatusUpdateEvent(
                        status=TaskStatus(
                            state=TaskState.input_required,
                            message=new_agent_text_message(
                                text_content,
                                task.contextId,
                                task.id,
                            ),
                        ),
                        final=True,
                        contextId=task.contextId,
                        taskId=task.id,
                    )
                )
            elif is_done:
                event_queue.enqueue_event(
                    TaskArtifactUpdateEvent(
                        append=False,
                        contextId=task.contextId,
                        taskId=task.id,
                        lastChunk=True,
                        artifact=new_text_artifact(
                            name='current_result',
                            description='Result of request to agent.',
                            text=text_content,
                        ),
                    )
                )
                event_queue.enqueue_event(
                    TaskStatusUpdateEvent(
                        status=TaskStatus(state=TaskState.completed),
                        final=True,
                        contextId=task.contextId,
                        taskId=task.id,
                    )
                )
            else:
                event_queue.enqueue_event(
                    TaskStatusUpdateEvent(
                        status=TaskStatus(
                            state=TaskState.working,
                            message=new_agent_text_message(
                                text_content,
                                task.contextId,
                                task.id,
                            ),
                        ),
                        final=False,
                        contextId=task.contextId,
                        taskId=task.id,
                    )
                )

    async def cancel(
        self, context: RequestContext, event_queue: EventQueue
    ) -> None:
        raise Exception('cancel not supported')


In [ ]:
import logging
import httpx

from starlette.applications import Starlette     # A2A wraps Starlette
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore, InMemoryPushNotifier
from a2a.types import AgentCapabilities, AgentCard, AgentSkill

# from a2a_agents import AbstractAgent            # your own base class
# from a2a_agent_exec import A2ALabAgentExecutor
# from a2a_agents import SemanticKernelAgent

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
# ------------------------------------------------------------------------

log = logging.getLogger(__name__)

TITLE                   = "Weather"
MCP_URL                 = "/weather/sse"
APIM_GATEWAY_URL        = "https://apim-ulp3pavelxqec.azure-api.net"
APIM_SUBSCRIPTION_KEY   = "**************"  # Keep it secret! Keep it safe!
OPENAI_API_VERSION      = "2024-10-21"
OPENAI_DEPLOYMENT_NAME  = "gpt-4o-mini"
ACA_URL                 = f"https://{os.environ.get('CONTAINER_APP_NAME', '')}.{os.environ.get('CONTAINER_APP_ENV_DNS_SUFFIX', '')}"
A2A_URL                 = "http://localhost:10020"

def build_app(
    *,
    host: str = "localhost",
    port: int = 10020,
) -> Starlette:
    """
    Assemble and return the fully-wired Starlette ASGI application.

    This function:
      • creates the Semantic Kernel agent
      • wraps it in an A2A executor
      • registers startup/shutdown hooks so the SSE socket is opened/closed
      • builds the A2A Starlette application and returns it
    """

    # -------- 1. Create the naked SemanticKernelAgent -----------------
    weather_agent = SemanticKernelAgent(
        mcp_url=f"{APIM_GATEWAY_URL}{MCP_URL}",
        title=TITLE,
        oai_client=AzureChatCompletion(
            endpoint=APIM_GATEWAY_URL,
            api_key=APIM_SUBSCRIPTION_KEY,
            api_version=OPENAI_API_VERSION,
            deployment_name=OPENAI_DEPLOYMENT_NAME,
        ),
    )

    # -------- 2. Wrap it in the A2A executor --------------------------
    sk_weather_agent_exec = A2ALabAgentExecutor(agent=weather_agent)

    # -------- 3. Wire the executor into the default request handler ---
    httpx_client   = httpx.AsyncClient()
    request_handler = DefaultRequestHandler(
        agent_executor = sk_weather_agent_exec,
        task_store     = InMemoryTaskStore(),
        push_notifier  = InMemoryPushNotifier(httpx_client),
    )

    # -------- 4. Build the A2A server via Starlette -------------------
    server = A2AStarletteApplication(
        agent_card   = _get_agent_card(host, port),
        http_handler = request_handler,
    )
    app: Starlette = server.build()

    # -------- 5. Register lifecycle hooks to open/close the agent -----
    @app.on_event("startup")
    async def _startup() -> None:
        log.info("Opening SemanticKernelAgent SSE connection …")
        await weather_agent.__aenter__()          # opens MCPSsePlugin
        # NB: if you decide to make the *executor* the context
        # manager (Option 2), just call `await sk_weather_agent_exec.__aenter__()`

    @app.on_event("shutdown")
    async def _shutdown() -> None:
        log.info("Closing SemanticKernelAgent SSE connection …")
        await weather_agent.__aexit__(None, None, None)
        await httpx_client.aclose()

    return app


# ========== Helper: build the agent-card sent to A2A clients ===========
def _get_agent_card(host: str, port: int) -> AgentCard:
    capabilities = AgentCapabilities(streaming=True)

    skill_weather = AgentSkill(
        id='weather_forecast_sk',
        name='Semantic Kernel Weather forecasting agent',
        description='Answers questions about the weather anywhere in the world',
        tags=['weather', 'semantic-kernel'],
        examples=[
            "What's the weather like in Cairo?",
            "Is it raining today in London and Paris?",
            "What's the capital of Sweden?",
        ],
    )

    return AgentCard(
        name='SK Weather Agent',
        description='Semantic-Kernel-powered weather agent',
        url=f'http://{host}:{port}/',
        version='1.0.0',
        defaultInputModes=['text'],
        defaultOutputModes=['text'],
        capabilities=capabilities,
        skills=[skill_weather],
    )

In [ ]:
# ------------------------------------------------------------------------
# run_server.py  – launch with `python run_server.py`
# ------------------------------------------------------------------------
import uvicorn, nest_asyncio

nest_asyncio.apply()

app = build_app()                  # Starlette ASGI application
uvicorn.run(app, host="0.0.0.0", port=10020)
